### Serial Liftout Data


In [ ]:
%load_ext autoreload
%autoreload 2

import glob
import os

import matplotlib.pyplot as plt
from random import shuffle



PATH =  "/home/patrick/github/data/liftout/serial-liftout/"
RAW_PATH = os.path.join(PATH, "raw/20230322_Cele_NK2476_B2_G1_SerLO/Movie")
DATA_PATH = os.path.join(PATH, "data")
RAW_PATH = os.path.join(DATA_PATH, "tmp")
TRAIN_PATH = os.path.join(DATA_PATH, "train")
TEST_PATH = os.path.join(DATA_PATH, "test")


os.makedirs(TRAIN_PATH, exist_ok=True)
os.makedirs(TEST_PATH, exist_ok=True)


filenames = glob.glob(os.path.join(RAW_PATH, "*Channel 2*.tif"))

# shuffle(filenames)

print(f"Found {len(filenames)} files")

In [ ]:

shuffle(filenames)

import tifffile as tff

shapes = {  }
new_filenames = glob.glob(os.path.join(TRAIN_PATH, "*.tif"))
processed = []
for i, fname in enumerate(filenames):
    # print(fname)

    basename = os.path.basename(fname)
    newfname = basename.split(" ")[-1].removesuffix(".tif")

    print(basename, newfname, f"{i:05d}")

    if newfname in new_filenames:
        print("DUPLICATE, SKIPPING")
        continue

    new_filenames.append(newfname)

    img = tff.imread(fname)

    CROP_PX = 45
    crop_img = img[:-CROP_PX, :]
    # print(basename, newfname, crop_img.shape)

    # # resize image to 1536x1024

    import cv2
    crop_img = cv2.resize(crop_img, (1536, 1024))

    # fig, ax = plt.subplots(1, 2, figsize=(20, 5))
    # ax[0].imshow(img, cmap="gray")
    # ax[1].imshow(crop_img, cmap="gray")
    # plt.show()


    if i <= 50:        
        tff.imwrite(os.path.join(TRAIN_PATH, f"{newfname}.tif"), crop_img)
        # print(i, "TRAIN")
    else:
        # print(i, "TEST")
        tff.imwrite(os.path.join(TEST_PATH, f"{newfname}.tif"), crop_img)

    processed.append(newfname)

    if i > 300:
        break

In [ ]:
print(shapes)

### EVALS


In [ ]:
filenames = sorted(glob.glob(os.path.join(TEST_PATH, "*.tif")))

EVAL_PATH = os.path.join(DATA_PATH, "eval/v3/final")
os.makedirs(EVAL_PATH, exist_ok=True)

print(f"Found {len(filenames)} files")

from fibsem.structures import FibsemImage
from fibsem.detection import detection
from fibsem.segmentation.model import load_model


CHECKPOINTS = [
    # {"checkpoint": "/home/patrick/github/fibsem/fibsem/segmentation/models/autoliftout/serial/base/v3/model04.pt", "encoder": "resnet34", "nc": 5},
    # {"checkpoint": "/home/patrick/github/fibsem/fibsem/segmentation/models/autoliftout/serial/base/v3/model14.pt", "encoder": "resnet34", "nc": 5},
    # {"checkpoint": "/home/patrick/github/fibsem/fibsem/segmentation/models/autoliftout/serial/base/v3/model19.pt", "encoder": "resnet34", "nc": 5},
    # {"checkpoint": "/home/patrick/github/fibsem/fibsem/segmentation/models/autoliftout/serial/base/v3/model24.pt", "encoder": "resnet34", "nc": 5},
    {"checkpoint": "/home/patrick/github/fibsem/fibsem/segmentation/models/autoliftout/serial/base/v3/model29.pt", "encoder": "resnet34", "nc": 5},
]

# shuffle(filenames)
for fname in filenames:
    
    img = FibsemImage.load(fname)

    features = [detection.LamellaCentre(), detection.NeedleTipBottom(), detection.CopperAdapterBottomEdge()]
    dets = []
    for i, ckpt in enumerate(CHECKPOINTS):

        model = load_model(checkpoint=ckpt["checkpoint"], encoder=ckpt["encoder"], nc=ckpt["nc"])

        det  = detection.detect_features(img.data, model, features=features, pixelsize=25e-9, filter=True)
        dets.append(det)
    
    print(fname)
    fig = detection.plot_detections(dets, titles=[os.path.basename(ckpt["checkpoint"]) for ckpt in CHECKPOINTS])

    # save to eval
    eval_fname = os.path.join(EVAL_PATH, os.path.basename(fname))
    # as png
    plt.savefig(eval_fname.removesuffix(".tif") + ".png", dpi=300, bbox_inches="tight") 
    # save figure
    plt.close(fig)

    # break
